In [ ]:
# ============================================================
# (3) 位相ズレ：相互相関でラグ推定（±max_lag）
# ============================================================
def best_lag_xcorr(x, t, max_lag=10):
    """
    x: 窓波形（1D）
    t: テンプレ（1D）
    戻り：lag（int）, score（正規化相互相関）
    """
    x = np.asarray(x, dtype=float).ravel()
    t = np.asarray(t, dtype=float).ravel()

    # 相互相関がオフセット(平均)に引っ張られないよう、平均を引く
    x0 = x - np.mean(x)
    t0 = t - np.mean(t)

    # full 相互相関（FFTで高速化）
    c = signal.correlate(x0, t0, mode="full", method="fft")
    lags = signal.correlation_lags(len(x0), len(t0), mode="full")

    # 探索範囲を ±max_lag に限定（実務では探索幅を狭めるのが定石）
    m = (lags >= -max_lag) & (lags <= max_lag)
    idx = int(np.argmax(c[m]))                            # 相互相関最大の位置
    lag = int(lags[m][idx])                               # そのラグ（サンプル単位）

    # 正規化相互相関（値域の目安を揃える）
    denom = (np.linalg.norm(x0) * np.linalg.norm(t0) + 1e-12)
    score = float(c[m][idx] / denom)

    return lag, score


def _align_by_lag(x, t, lag):
    """
    推定ラグに合わせて「重なる部分」だけ切り出す（循環シフトはしない）
    """
    if lag > 0:
        return x[lag:], t[:-lag]
    if lag < 0:
        return x[:lag], t[-lag:]
    return x, t


# ============================================================
# (2) 波形の変化（形）：テンプレ比較（相関・誤差）
# (3) 位相ズレ：phase_lag と xcorr_score
# ============================================================
def feat_template_one(w, tmpl, max_lag=10):
    x = _fill_nan_1d(w)
    t = _fill_nan_1d(tmpl)

    # 形状比較のため、オフセットは外す（オフセットは(4)で別特徴）
    x0 = x - np.mean(x)
    t0 = t - np.mean(t)

    lag, xcorr_score = best_lag_xcorr(x0, t0, max_lag=max_lag)
    xa, ta = _align_by_lag(x0, t0, lag)

    # データが短すぎると相関や誤差が不安定なので、0で返す（NaN回避）
    if xa.size < 3 or ta.size < 3:
        return {
            "phase_lag": int(lag),
            "xcorr_score": float(xcorr_score),
            "corrcoef_aligned": 0.0,
            "mse": 0.0, "rmse": 0.0, "nrmse": 0.0, "mae": 0.0
        }

    # 相関係数（形がどれだけ似ているか）
    if np.std(xa) < 1e-12 or np.std(ta) < 1e-12:
        corrcoef = 0.0
    else:
        corrcoef = float(np.corrcoef(xa, ta)[0, 1])

    # 誤差（どれだけ違うか）
    diff = xa - ta
    mse = float(np.mean(diff**2))
    rmse = float(np.sqrt(mse))
    mae = float(np.mean(np.abs(diff)))

    # NRMSE（テンプレのレンジで正規化）
    denom = float(np.ptp(ta))
    nrmse = float(rmse / (denom + 1e-12)) if denom > 0 else 0.0

    return {
        "phase_lag": int(lag),
        "xcorr_score": float(xcorr_score),
        "corrcoef_aligned": corrcoef,
        "mse": mse, "rmse": rmse, "nrmse": nrmse, "mae": mae
    }



## 2つの関数の役割（混乱ポイントの整理）

`best_lag_xcorr(x, t)` は「横ずれ（何サンプルずれているか）」を推定する関数です。返すのは `lag` と、その `lag` のときの「どれくらい重なるか」の点数 `score` だけです。

`feat_template_one(w, tmpl)` は `best_lag_xcorr` を内部で呼んで `lag` をまず推定し、そのあと「ずれを直して重ねた状態（aligned）」で、形の違いをいくつかの指標（`corrcoef_aligned / mse / rmse / nrmse / mae`）で測って返す関数です。

結論として  
- `lag` は「位置のズレ量」  
- `corrcoef_aligned / mse / rmse / nrmse / mae` は「位置を合わせた後でも形がどれだけ違うか」  
を見ています。だから両方が必要です（ズレだけなら合わせれば形は同じになるが、形が変形していたら合わせても違いが残る）。

---

## 指標の定義（数式）

`_align_by_lag` 後に得られる重なり部分を `x_a, t_a`（同じ長さ N）とし、差を `d_i = x_a[i] - t_a[i]` とします。

### corrcoef_aligned（相関係数）
形が「同じ方向に増減しているか」を見る無次元の値。だいたい -1 〜 1。  
重要：振幅が2倍になっても形が同じなら 1 に近くなりやすい。

$$
\mathrm{corr}(x_a,t_a)=\frac{\sum_{i=1}^{N}(x_a[i]-\bar{x}_a)(t_a[i]-\bar{t}_a)}
{\sqrt{\sum_{i=1}^{N}(x_a[i]-\bar{x}_a)^2}\sqrt{\sum_{i=1}^{N}(t_a[i]-\bar{t}_a)^2}}
$$

### mse（平均二乗誤差）
差を二乗して平均。大きいズレ（スパイク）を強く罰する。単位は「信号の単位の二乗」。

$$
\mathrm{MSE}=\frac{1}{N}\sum_{i=1}^{N} d_i^2
$$

### rmse（二乗平均平方根誤差）
mse の平方根。単位が元の信号と同じで直感的。

$$
\mathrm{RMSE}=\sqrt{\mathrm{MSE}}
$$

### mae（平均絶対誤差）
差の絶対値の平均。mse よりスパイクに引っ張られにくい。単位は元の信号と同じ。

$$
\mathrm{MAE}=\frac{1}{N}\sum_{i=1}^{N} |d_i|
$$

### nrmse（正規化RMSE）
rmse を「テンプレの振れ幅」で割って無次元化。センサ間比較をしやすくする。  
このコードではテンプレ側のレンジ（最大−最小）で割る。

$$
\mathrm{NRMSE}=\frac{\mathrm{RMSE}}{\max(t_a)-\min(t_a)+\epsilon}
$$

---

## 具体例で「違い」を一発で理解する

### 例1：形は同じだが 2倍に伸びた（aligned 後）
t_a = [0,1,2]  
x_a = [0,2,4]  
差 d = [0,1,2]

- corrcoef_aligned = 1.0（形の増減が完全一致）
- MSE = (0^2+1^2+2^2)/3 = 5/3 = 1.6667
- RMSE = sqrt(1.6667) = 1.2910
- MAE = (|0|+|1|+|2|)/3 = 1.0000
- テンプレのレンジ ptp = 2 → NRMSE = 1.2910/2 = 0.6455

ここから分かること：  
相関（corrcoef）は「形」を見るので 1.0 になりやすい。  
mse/rmse/mae は「量のズレ」を見るので大きくなる。

### 例2：スパイク1点だけ大外れ（aligned 後）
t_a = [0,1,2]  
x_a = [0,1,10]  
差 d = [0,0,8]

- MSE = (64)/3 = 21.3333（爆発的に大きい）
- MAE = 8/3 = 2.6667（mseほどは増えない）

ここから分かること：  
mse/rmse はスパイクに敏感。  
mae は比較的頑健。だから両方あると状況分けができる。

---

## best_lag_xcorr の score と corrcoef_aligned の違い（ここが重要）

best_lag_xcorr の score は「あるラグでの相互相関」をノルムで正規化した値です。  
この実装では分母が “全長のノルム” 基準なので、ラグが大きくなって重なりが減ると score が下がりやすい性質があります。

一方 corrcoef_aligned は「ラグに合わせて切り出した重なり部分だけ」で相関を取ります。  
つまり  
- score：重なりが少ない（信頼が薄い）ことも自然に罰する  
- corrcoef_aligned：重なった範囲で形が似ているかを素直に見る  
という違いです。

---